In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import torch
from tqdm import tqdm

In [20]:
start = time.time()
df = pd.read_csv('cleaned_puzzle_batches/cleaned_train_batch_1.csv')

output_df = df.filter(like='theme')
input_cols = [col_name for col_name in df.columns if not col_name.startswith('theme')]
input_df = df.filter(items=input_cols)

corr_df = output_df.apply(lambda x: input_df.corrwith(x))

end = time.time()
duration = end - start
print("exec time: %.4fs" % duration)

exec time: 6.9935s


In [21]:
torch.cuda.is_available()

True

In [22]:
class ChessData(torch.utils.data.Dataset):
    def __init__(self, path):
        self.df = pd.read_csv(path)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = df.iloc[idx]

        inputs = row[:321].values.astype(np.float32)
        outputs = row[321:].values.astype(np.float32)

        return {'board': inputs, 'themes': outputs}


In [23]:
from torch.utils.data import DataLoader

chess_data_set = ChessData('cleaned_puzzle_batches/cleaned_train_batch_1.csv')

train_dataloader = DataLoader(chess_data_set, batch_size=64, shuffle=True)
print(train_dataloader)

# Display image and label.
batch = next(iter(train_dataloader))
#print(f"batch['board']: {batch['board']}")
#print(f"batch['themes']: {batch['themes']}")
#print(f"batch['themes']: {batch['themes'].shape}")

In [24]:
import torch.nn as nn

net = nn.Sequential(nn.Linear(321, 27))


In [25]:
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
net = net.cuda()

for epoch in range(1, 101):
    for batch in tqdm(train_dataloader):
        X = batch['board'].cuda()

        pred = net(X)
        loss = loss_fn(pred, batch['themes'].cuda())

        optim.zero_grad()
        loss.backward()

        optim.step()

